In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
from feature_utils import generate_sdf_from_smiles_using_rdkit

info = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/info_core_pockets.csv")
kekule_smiles = list(info.kekule_smiles.unique())
smiles_names = list(info.smiles_name.unique())
smiles_dict = torch.load("/home/jovyan/dataspace/Prototype/smiles_dict.pt")

KeyboardInterrupt: 

根据 `kekule_smiles` 生成小分子文件。只需要执行一次。
这里或许应该生成一份，shifted 的小分子文件！！！

In [ ]:
if True:
    error_smiles = []
    for i, smiles in tqdm(enumerate(kekule_smiles), total=len(kekule_smiles)):
        try:
            smiles_name = smiles_dict[smiles]
            generate_sdf_from_smiles_using_rdkit(smiles=smiles, rdkitMolFile=f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_shifted/{smiles_name}.sdf", shift_dis=30)
        except:
            error_smiles.append((smiles, smiles_dict[smiles]))

In [ ]:
from feature_utils import extract_torchdrug_feature_from_mol
from feature_utils import read_mol

compound_dict = {}
skip_smiles_list = []
for smiles_name in tqdm(smiles_names):
    mol, _ = read_mol(f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_shifted/{smiles_name}.sdf", None)
    # extract features from sdf.
    try:
        compound_dict[smiles_name] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)  # self-dock set has_LAS_mask to true
    except Exception as e:
        print(smiles_name, e)
        skip_smiles_list.append(smiles_name)

skip_smiles_list = ['smiles_5544', 'smiles_39785', 'smiles_39786', 'smiles_39787', 'smiles_39788', 'smiles_40760', 'smiles_40762', 'smiles_40763', 'smiles_40764', 'smiles_40766', 'smiles_43789']
tankbind_data_path = f"/home/jovyan/dataspace/Prototype/tankbind_data"
torch.save(compound_dict, f"{tankbind_data_path}/compound_torchdrug_features_shifted.pt")